In [115]:
library('data.table')
library('dplyr')
library('tidyr')
library('ggplot2')
library('scales')

In [116]:
titles <- fread("title_ratings", quote="", na.strings = "\\N")
aka <- fread("aka_filtered", quote = "", na.strings = "\\N")
crew <- fread("crew_filtered", na.strings = "\\N")
principals <- fread("principals_filtered", quote="", na.strings = "\\N")
episodes <- fread("episodes_filtered", na.strings = "\\N")
names <-fread("names_filtered", na.strings = "\\N")

In [117]:
# titles and their ratings
head(titles, 3)
# the names of the titles in different languages, probably not relevant
head(aka, 3)
# list of writers and directors for each movie
head(crew, 3)
# list of roles of an actor in the given movie
head(principals, 3)
# table tying episodes to its tvSeries
head(ep_series, 3)
# names of the actors with the titles they starred in
head(names, 3)

tconst,titleType,primaryTitle,originalTitle,isAdult,runtimeMinutes,genres,year,averageRating,numVotes
<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<chr>,<int>,<dbl>,<int>
tt0043224,tvSeries,The Red Skelton Hour,The Red Skelton Show,FALSE,60,Comedy,2016,8.1,1285
tt0044265,tvSeries,Guiding Light,The Guiding Light,FALSE,60,"Drama,Romance",2009,6.4,1722
tt0048845,tvSeries,As the World Turns,As the World Turns,FALSE,60,"Drama,Romance",2010,6.1,2102


titleId,title,region,language,types,isOriginalTitle
<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
tt0043224,The Red Skelton Hour,US,NA,NA,FALSE
tt0043224,The Red Skelton Show,CA,en,imdbDisplay,FALSE
tt0043224,Шоу Рэда Скелтона,SUHH,ru,imdbDisplay,FALSE


tconst,directors,writers
<chr>,<chr>,<chr>
tt0043224,"nm0477438,nm0054846,nm1096821,nm0705228,nm0120437,nm0076971,nm0387695,nm0232902,nm1097266,nm1637827","nm0804026,nm0639495,nm0680226,nm0777442,nm0706417,nm0722289,nm1640023,nm0338876,nm0326239,nm0777129,nm0414633,nm0288997,nm1520544,nm0799745,nm0615080,nm0293159,nm0771627,nm0918826,nm0410536,nm2204703,nm0397170,nm0917200,nm0594174,nm0917026,nm0566588,nm0288816,nm3123274,nm0051332,nm2964448,nm0237341,nm0329085,nm1034110,nm3110698,nm0516015,nm0057677,nm0578441,nm0814068,nm0081748,nm0606806,nm0639787,nm0432188,nm0498586,nm0743232,nm2198070,nm0743259,nm0777202,nm0380890,nm2915847,nm0377958,nm1290769,nm7410201,nm0545131,nm0655462,nm3201019,nm0001992,nm0399849,nm0000036"
tt0044265,"nm0004098,nm0179413,nm1597578,nm1335675,nm0915727,nm0581589,nm0568114,nm0213789,nm0281224,nm0332752,nm0884037,nm0297489,nm0109586,nm0331135,nm1513459,nm0778479,nm0224683,nm0159012,nm0100841,nm0250864,nm2681961,nm1273119,nm0923396,nm0369858,nm2036776,nm1334802,nm1603516,nm1314913,nm1343326,nm0925870,nm0834419,nm2552851,nm3174789,nm0963061,nm0571983,nm0322900","nm2419908,nm1141830,nm2251709,nm2078312,nm1003809,nm0325029,nm1014203,nm1014177,nm0492953,nm1012482,nm1010971,nm0519133,nm0174585,nm0429390,nm2429218,nm2654235,nm1221904,nm0068550,nm1159577,nm0909529,nm3049540,nm0255819,nm0260547,nm0113780,nm0628248,nm0717622,nm0989904,nm1142664,nm0191612,nm1051829,nm2727176,nm4972117,nm0997483,nm0110828,nm1105215,nm3079954,nm1779986,nm2085815,nm0693044,nm0106187,nm0322325,nm0833510,nm0807006,nm0542021,nm3454108,nm0566699,nm2721275,nm4237316,nm0291507,nm0027442,nm1267575,nm1011786,nm0750422,nm1014193,nm0820121,nm1011976,nm1000132,nm1007597,nm0792404,nm1049086,nm0996693,nm1088908,nm0229890,nm0229908,nm0913330,nm0758444,nm3059609,nm0782462,nm0746169,nm0925335,nm0925388,nm0501148,nm2040174,nm0711626,nm1480454,nm0421246,nm0846520,nm0922948,nm2591020,nm0925872,nm1012917,nm9109568,nm0617002,nm0682274,nm3566933,nm0723559,nm1779949,nm0658855,nm0880986,nm1276101,nm1299367,nm0434560,nm0115032,nm1011004,nm0358462,nm2272293,nm0906799,nm0479525,nm0997301,nm3579833,nm0534951,nm1802728,nm0680460,nm2654307,nm0866801,nm2660138,nm1669530,nm0040612,nm0481805,nm1106086,nm0899533,nm2077775,nm2220634,nm1049067,nm2006443,nm0812026,nm0111576,nm0698989,nm0019706,nm0111577,nm0122417,nm0202681,nm0238898,nm0505615,nm0754581,nm0789737,nm0991335,nm0162914,nm1301702,nm3059996,nm0726721,nm1014206,nm2233247,nm0235902,nm10360337,nm1672253,nm1440687,nm1400899,nm5846217,nm0817957,nm2076865,nm9551033,nm3930118,nm0068589,nm0633202,nm0881501,nm0813960,nm0989955,nm0031021,nm0097597,nm0112601,nm0148223,nm0313072,nm0368575,nm0513851,nm0564474"
tt0048845,"nm0332752,nm0824738,nm0100841,nm0179413,nm0251732,nm0887790,nm4029369,nm0245698,nm0777413,nm0114113,nm0157553,nm0384316,nm0906004,nm0041715,nm0219989,nm0242409,nm1681063,nm1212523,nm0856388,nm0004098,nm0594204,nm0046490,nm1408290,nm0446060,nm0483582,nm0357800,nm0639479,nm0581589,nm2483215,nm0213789,nm0017875,nm0802986,nm0295925,nm1656173,nm0884037,nm1032520,nm0591523,nm1010949,nm0923396,nm0672625,nm0867723,nm1603516,nm0109586,nm0571983,nm0677210,nm0139400,nm0656440,nm0125226,nm0672627,nm0641266,nm0723593,nm0963061","nm0664799,nm1923276,nm1007597,nm2419908,nm1010949,nm3049540,nm1779949,nm1779986,nm1049066,nm0680460,nm0633202,nm0790760,nm1014174,nm1299367,nm1014216,nm0045741,nm1014167,nm1011004,nm0481805,nm0909529,nm1014175,nm0106187,nm0787317,nm0655491,nm1780023,nm1011440,nm0792404,nm1010971,nm1141830,nm1400899,nm0153944,nm1014176,nm0617002,nm1014164,nm1014206,nm0429390,nm0507406,nm0925872,nm0713131,nm0108338,nm0191612,nm1042591,nm1010857,nm0632525,nm1088908,nm1892355,nm0325029,nm1250955,nm0996693,nm1104059,nm0255034,nm0110828,nm2812299,nm2663899,nm0335072,nm1045621,nm0494959,nm0711626,nm0085519,nm1250889,nm0827711,nm1142664,nm1113445,nm10285461,nm1014207,nm1715559,nm1809263,nm1012917,nm2591020,nm1276101,nm0071291,nm1049067,nm2373457,nm1030464,nm1000124,nm3113329,nm1431618,nm1051829,nm0311869,nm1089437,nm04

tconst,ordering,nconst,category,job,characters
<chr>,<int>,<chr>,<chr>,<chr>,<chr>
tt0043224,10,nm0461246,self,NA,"[""Self - Commercials"",""Commercials"",""Blonde Nurse-Hospital Sketch""]"
tt0043224,1,nm0804026,self,NA,"[""Self - Host"",""Clem Kadiddlehopper"",""Freddie the Freeloader""]"
tt0043224,2,nm2582914,self,NA,"[""Themselves"",""Orchestra"",""Singers""]"


tconst,parentTconst,seasonNumber,episodeNumber
<chr>,<chr>,<chr>,<chr>
tt0182919,tt1466074,13,4
tt0187660,tt0086779,10,1
tt0199398,tt0920489,2,2


nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
<chr>,<chr>,<int>,<int>,<chr>,<chr>
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0031983,tt0072308"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0071877"
nm0000003,Brigitte Bardot,1934,NA,"actress,soundtrack,music_department","tt0056404,tt0049189,tt0054452,tt0057345"


In [118]:
head(ep_series, 3)
count(ep_series)

tconst,parentTconst,seasonNumber,episodeNumber
<chr>,<chr>,<chr>,<chr>
tt0182919,tt1466074,13,4
tt0187660,tt0086779,10,1
tt0199398,tt0920489,2,2


n
<int>
262703


## Further filtering

### Series episodes split

In [119]:
series <- titles %>% filter(titleType != 'tvEpisode')
episodes <- titles %>% filter(titleType == 'tvEpisode')
count(series)
count(episodes)

n
<int>
77090


n
<int>
459147


### Filter tvSeries under 1000 reviews and their related elements

In [120]:
series <- series %>% filter(numVotes >= 1000)

In [121]:
ep_series <- ep_series %>% filter(parentTconst %in% series$tconst)

In [122]:
episodes <- episodes %>% filter(tconst %in% ep_series$tconst)

In [123]:
principals <- principals %>% filter(tconst %in% series$tconst | tconst %in% episodes$tconst)
names_filtered <- data_names %>% filter(nconst %in% principals$nconst)
aka_filtered <- data_aka %>% filter(titleId %in% series$tconst | titleId %in% episodes$tconst)
crew_filtered <- data_crew %>% filter(tconst %in% series$tconst | tconst %in% episodes$tconst)

### Data transformations

In [124]:
# series and episodes, - split genres, type -> factor
series$titleType = as.factor(series$titleType)
series <- series %>% mutate(genres = strsplit(genres, split=','))
episodes$titleType = as.factor(episodes$titleType)
episodes <- episodes %>% mutate(genres = strsplit(genres, split=','))
head(episodes)

tconst,titleType,primaryTitle,originalTitle,isAdult,runtimeMinutes,genres,year,averageRating,numVotes
<chr>,<fct>,<chr>,<chr>,<lgl>,<int>,<list>,<int>,<dbl>,<int>
tt0182919,tvEpisode,Murder with Too Many Notes,Murder with Too Many Notes,FALSE,98,"Crime , Drama , Mystery",2000,6.7,1666
tt0187660,tvEpisode,The Octopus 10,La piovra 10,FALSE,102,"Crime , Drama , Mystery",2001,7.3,295
tt0199398,tvEpisode,The Terracotta Dog,Il cane di terracotta,FALSE,102,"Crime , Drama , Mystery",2000,7.6,281
tt0199536,tvEpisode,The Shape of Water,La forma dell'acqua,FALSE,112,"Crime , Drama , Mystery",2000,7.5,386
tt0221289,tvEpisode,Pilot,Pilot,FALSE,120,"Action, Comedy, Sci-Fi",2000,7.6,468
tt0235326,tvEpisode,Pilot,Pilot,FALSE,87,"Action , Drama , Mystery",2000,7.3,2427


In [125]:
# crew - split directors and writers
crew <- crew %>% mutate(directors = strsplit(directors, split = ','), writers = strsplit(writers, split = ','))
head(crew, 3)

tconst,directors,writers
<chr>,<list>,<list>
tt0043224,"nm0477438, nm0054846, nm1096821, nm0705228, nm0120437, nm0076971, nm0387695, nm0232902, nm1097266, nm1637827","nm0804026, nm0639495, nm0680226, nm0777442, nm0706417, nm0722289, nm1640023, nm0338876, nm0326239, nm0777129, nm0414633, nm0288997, nm1520544, nm0799745, nm0615080, nm0293159, nm0771627, nm0918826, nm0410536, nm2204703, nm0397170, nm0917200, nm0594174, nm0917026, nm0566588, nm0288816, nm3123274, nm0051332, nm2964448, nm0237341, nm0329085, nm1034110, nm3110698, nm0516015, nm0057677, nm0578441, nm0814068, nm0081748, nm0606806, nm0639787, nm0432188, nm0498586, nm0743232, nm2198070, nm0743259, nm0777202, nm0380890, nm2915847, nm0377958, nm1290769, nm7410201, nm0545131, nm0655462, nm3201019, nm0001992, nm0399849, nm0000036"
tt0044265,"nm0004098, nm0179413, nm1597578, nm1335675, nm0915727, nm0581589, nm0568114, nm0213789, nm0281224, nm0332752, nm0884037, nm0297489, nm0109586, nm0331135, nm1513459, nm0778479, nm0224683, nm0159012, nm0100841, nm0250864, nm2681961, nm1273119, nm0923396, nm0369858, nm2036776, nm1334802, nm1603516, nm1314913, nm1343326, nm0925870, nm0834419, nm2552851, nm3174789, nm0963061, nm0571983, nm0322900","nm2419908 , nm1141830 , nm2251709 , nm2078312 , nm1003809 , nm0325029 , nm1014203 , nm1014177 , nm0492953 , nm1012482 , nm1010971 , nm0519133 , nm0174585 , nm0429390 , nm2429218 , nm2654235 , nm1221904 , nm0068550 , nm1159577 , nm0909529 , nm3049540 , nm0255819 , nm0260547 , nm0113780 , nm0628248 , nm0717622 , nm0989904 , nm1142664 , nm0191612 , nm1051829 , nm2727176 , nm4972117 , nm0997483 , nm0110828 , nm1105215 , nm3079954 , nm1779986 , nm2085815 , nm0693044 , nm0106187 , nm0322325 , nm0833510 , nm0807006 , nm0542021 , nm3454108 , nm0566699 , nm2721275 , nm4237316 , nm0291507 , nm0027442 , nm1267575 , nm1011786 , nm0750422 , nm1014193 , nm0820121 , nm1011976 , nm1000132 , nm1007597 , nm0792404 , nm1049086 , nm0996693 , nm1088908 , nm0229890 , nm0229908 , nm0913330 , nm0758444 , nm3059609 , nm0782462 , nm0746169 , nm0925335 , nm0925388 , nm0501148 , nm2040174 , nm0711626 , nm1480454 , nm0421246 , nm0846520 , nm0922948 , nm2591020 , nm0925872 , nm1012917 , nm9109568 , nm0617002 , nm0682274 , nm3566933 , nm0723559 , nm1779949 , nm0658855 , nm0880986 , nm1276101 , nm1299367 , nm0434560 , nm0115032 , nm1011004 , nm0358462 , nm2272293 , nm0906799 , nm0479525 , nm0997301 , nm3579833 , nm0534951 , nm1802728 , nm0680460 , nm2654307 , nm0866801 , nm2660138 , nm1669530 , nm0040612 , nm0481805 , nm1106086 , nm0899533 , nm2077775 , nm2220634 , nm1049067 , nm2006443 , nm0812026 , nm0111576 , nm0698989 , nm0019706 , nm0111577 , nm0122417 , nm0202681 , nm0238898 , nm0505615 , nm0754581 , nm0789737 , nm0991335 , nm0162914 , nm1301702 , nm3059996 , nm0726721 , nm1014206 , nm2233247 , nm0235902 , nm10360337, nm1672253 , nm1440687 , nm1400899 , nm5846217 , nm0817957 , nm2076865 , nm9551033 , nm3930118 , nm0068589 , nm0633202 , nm0881501 , nm0813960 , nm0989955 , nm0031021 , nm0097597 , nm0112601 , nm0148223 , nm0313072 , nm0368575 , nm0513851 , nm0564474"
tt0048845,"nm0332752, nm0824738, nm0100841, nm0179413, nm0251732, nm0887790, nm4029369, nm0245698, nm0777413, nm0114113, nm0157553, nm0384316, nm0906004, nm0041715, nm0219989, nm0242409, nm1681063, nm1212523, nm0856388, nm0004098, nm0594204, nm0046490, nm1408290, nm0446060, nm0483582, nm0357800, nm0639479, nm0581589, nm2483215, nm0213789, nm0017875, nm0802986, nm0295925, nm1656173, nm0884037, nm1032520, nm0591523, nm1010949, nm0923396, nm0672625, nm0867723, nm1603516, nm0109586, nm0571983, nm0677210, nm0139400, nm0656440, nm0125226, nm0672627, nm0641266, nm0723593, nm0963061","nm0664799 , nm1923276 , nm1007597 , nm2419908 , nm1010949 , nm3049540 , nm1779949 , nm1779986 , nm1049066 , nm0680460 , nm0633202 , nm0790760 , nm1014174 , nm1299367 , nm1014216 , nm0045741 , nm1014167 , nm1011004 , nm0481805 , nm0909529 , nm1014175 , nm0106187 , nm0787317 , nm0655491 , nm1780023 , nm1011440 , nm0792404 , nm1010971 , nm11418

In [ ]:
# principals - delete brackets from characters and split, catogry -> factor
principals$category <- as.factor(principals$category)
principals$characters <- gsub("\\]", '', principals$characters)
principals$characters <- gsub("\\[", '', principals$characters)
principals$characters <- gsub('"', '', principals$characters)
principals$characters <- gsub('"', '', principals$characters)
principals <- principals %>% mutate(characters = strsplit(characters, split = ','))
head(principals, 3)

In [146]:
# ep_series - season and ep number -> int
ep_series$seasonNumber <- as.integer(ep_series$seasonNumber)
ep_series$episodeNumber <- as.integer(ep_series$episodeNumber)
head(ep_series, 3)

tconst,parentTconst,seasonNumber,episodeNumber
<chr>,<chr>,<int>,<int>
tt0182919,tt1466074,13,4
tt0187660,tt0086779,10,1
tt0199398,tt0920489,2,2


In [148]:
# names - split proffesion and titles
names <- names %>% mutate(primaryProfession = strsplit(primaryProfession, split = ','), knownForTitles = strsplit(knownForTitles, split = ','))
head(names, 3)

nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
<chr>,<chr>,<int>,<int>,<list>,<list>
nm0000001,Fred Astaire,1899,1987,"soundtrack , actor , miscellaneous","tt0050419, tt0053137, tt0031983, tt0072308"
nm0000002,Lauren Bacall,1924,2014,"actress , soundtrack","tt0117057, tt0037382, tt0038355, tt0071877"
nm0000003,Brigitte Bardot,1934,NA,"actress , soundtrack , music_department","tt0056404, tt0049189, tt0054452, tt0057345"


In [149]:
# free up memory after transformations
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,9446282,504.5,17042405,910.2,14159992,756.3
Vcells,208694377,1592.3,307424909,2345.5,307424903,2345.5
